Paint con manos

In [2]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import math
import time


In [8]:
def add_points_to_list(list_of_draws, hand_landmarks, point_color):
  thumb_x = hand_landmarks.landmark[4].x 
  thumb_y = hand_landmarks.landmark[4].y
  thumb_z = hand_landmarks.landmark[4].z 

  index_x = hand_landmarks.landmark[8].x
  index_y = hand_landmarks.landmark[8].y 
  index_z = hand_landmarks.landmark[8].z
  
  # Distancia entre la punta del pulgar y la del índice
  distance = math.sqrt((thumb_x - index_x)**2 + (thumb_y - index_y)**2)

  # Umbral de cercania
  threshold = 0.07

  if distance < threshold:
      midpoint = ((thumb_x + index_x) / 2, (thumb_y + index_y) / 2, (thumb_z + index_z) / 2, point_color)
      list_of_draws.append(midpoint)

  # Añadir una posicion sin uso (-1, -1, -1) para cuando no se este dibujando
  elif list_of_draws and list_of_draws[-1][0] != -1: 
    list_of_draws.append((-1, -1, -1, (255, 255, 255)))



def draw_lines(list_of_draws, image):
    # Funcion para juntar todos los puntos que se han creado de forma consecutiva para crear el dibujo
    x, y, z = (-1, -1, -1)

    for position in list_of_draws:
      prev_x, prev_y, prev_z = x, y, z
      x, y, z, color_point = position

      # No unir puntos si se paro de dibujar
      if (x, y, z) == (-1, -1, -1) or (prev_x == -1 and prev_y == -1 and prev_z == -1):
        continue

      height, width, deep = image.shape
      midpoint_x, midpoint_y, midpoint_z= int(x * width), int(y * height), int(z * deep)
      prev_midpoint_x, prev_midpoint_y, prev_midpoint_z= int(prev_x * width), int(prev_y * height), int(prev_z * deep)
      
      # Evitar dibujar sobre la paleta
      if 190 < prev_midpoint_x < 460 and 190 < midpoint_x < 460 and 0 < prev_midpoint_y < 90 and 0 < midpoint_y < 90:
        continue 
      
      cv2.line(image, (midpoint_x, midpoint_y), (prev_midpoint_x, prev_midpoint_y), color_point, 5)

In [9]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Paleta de colores
paleta = cv2.imread("media/fotos/paleta.png") 
paleta = cv2.resize(paleta, (250, 70))

# Variable para añadir delay al cambiar color
last_d_key_press_time = time.time()

# Diccionario con los colores, su codigo bgr y su posicion en el frame
colors_dict = {
    'red': ((0, 0, 248), (204, 15)),
    'orange': ((0, 102, 255), (204, 48)),
    'salmon': ((104, 0, 255), (235, 15)),
    'pink': ((210, 0, 254), (235, 48)),
    'purple': ((176, 0, 161), (265, 15)),
    'deep purple': ((165, 40, 102), (265, 48)),
    'deep blue': ((206, 52, 53), (295, 15)),
    'blue': ((216, 172, 1), (295, 48)),
    'green': ((0, 172, 0), (326, 15)),
    'soft blue': ((137, 161, 1), (326, 48)),
    'yellow': ((0, 205, 255), (356, 15)),
    'deep green': ((0, 213, 150), (356, 48)),
    'almost black': ((34, 5, 36), (387, 15)),
    'soft pink': ((243, 192, 247), (387, 48)),
    'black': ((0, 0, 0), (417, 15)),
    'white': ((255, 255, 255), (417, 48))
}

# Variables para manejar el color actual
color_names = list(colors_dict.keys())
current_color_index = 0

# Inicializar clase Hands
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    #max_num_hands=2,
    min_tracking_confidence=0.5) as hands:
  

  # Lista para almacenar las posiciones en donde se dibuja con su color  
  list_of_draws = []

  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      continue

    # Almacenar los datos del detector de manos sobre el frame
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)   
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Colocar la paleta en el frame
    image[10:10 +70, 200:200 +250] = paleta
    color_name = color_names[current_color_index]


    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style()
        )
        add_points_to_list(list_of_draws, hand_landmarks, colors_dict[color_name][0])
    else:
       if list_of_draws and list_of_draws[-1][0] != -1: 
          list_of_draws.append((-1, -1, -1, (255, 255, 255)))
          
    draw_lines(list_of_draws, image)
  


    # Cuadrado que muestra el color actual  
    x_color_square, y_color_square = colors_dict[color_name][1]  
    cv2.rectangle(image, (x_color_square, y_color_square), (x_color_square+27, y_color_square+27), (162, 0, 0), 2)

    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))



    # Si se pulsa 'z' deshacer lo que se ha dibujado    
    key = cv2.waitKey(1) & 0xFF

    if key  == ord('z'):
      while list_of_draws and list_of_draws[-1][0] == -1:
        list_of_draws.pop()
      
      if list_of_draws:
        list_of_draws.pop() 

    elif key == ord('b') and list_of_draws:  # Tecla 'z'
        print("Funciona 'b'")
        if list_of_draws[-1][0] == -1:
            list_of_draws.pop()
        while list_of_draws and list_of_draws[-1][0] != -1:
            list_of_draws.pop()

    # Si se pulsa 'd' cambiar de color
    elif key  == ord('d'):
      current_time = time.time()
      if current_time - last_d_key_press_time >= 1.1:
          current_color_index = (current_color_index +1) % len(color_names)
          last_d_key_press_time = current_time

    # Terminar si se pulsa 'q'
    elif cv2.waitKey(1) & 0xFF == ord('q'):
      break
cap.release()
cv2.destroyAllWindows()

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Paleta de colores
paleta = cv2.imread("media/fotos/paleta.png") 
paleta = cv2.resize(paleta, (250, 70))

# Variable para añadir delay al cambiar color
last_d_key_press_time = time.time()

# Diccionario con los colores, su codigo bgr y su posicion en el frame
colors_dict = {
    'red': ((0, 0, 248), (204, 15)),
    'orange': ((0, 102, 255), (204, 48)),
    'salmon': ((104, 0, 255), (235, 15)),
    'pink': ((210, 0, 254), (235, 48)),
    'purple': ((176, 0, 161), (265, 15)),
    'deep purple': ((165, 40, 102), (265, 48)),
    'deep blue': ((206, 52, 53), (295, 15)),
    'blue': ((216, 172, 1), (295, 48)),
    'green': ((0, 172, 0), (326, 15)),
    'soft blue': ((137, 161, 1), (326, 48)),
    'yellow': ((0, 205, 255), (356, 15)),
    'deep green': ((0, 213, 150), (356, 48)),
    'almost black': ((34, 5, 36), (387, 15)),
    'soft pink': ((243, 192, 247), (387, 48)),
    'black': ((0, 0, 0), (417, 15)),
    'white': ((255, 255, 255), (417, 48))
}

# Variables para manejar el color actual
color_names = list(colors_dict.keys())
current_color_index = 0

# Inicializar clase Hands
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    #max_num_hands=2,
    min_tracking_confidence=0.5) as hands:
  

  # Lista para almacenar las posiciones en donde se dibuja con su color  
  list_of_draws = []

  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      continue

    # Almacenar los datos del detector de manos sobre el frame
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)   
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Colocar la paleta en el frame
    image[10:10 +70, 200:200 +250] = paleta
    color_name = color_names[current_color_index]


    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style()
        )
        add_points_to_list(list_of_draws, hand_landmarks, colors_dict[color_name][0])
    else:
       if list_of_draws and list_of_draws[-1][0] != -1: 
          list_of_draws.append((-1, -1, -1, (255, 255, 255)))
          
    draw_lines(list_of_draws, image)
  


    # Cuadrado que muestra el color actual  
    x_color_square, y_color_square = colors_dict[color_name][1]  
    cv2.rectangle(image, (x_color_square, y_color_square), (x_color_square+27, y_color_square+27), (162, 0, 0), 2)

    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))



    # Si se pulsa 'z' deshacer lo que se ha dibujado    
    key = cv2.waitKey(1) & 0xFF

    if key  == ord('z'):
      while list_of_draws and list_of_draws[-1][0] == -1:
        list_of_draws.pop()
      
      if list_of_draws:
        list_of_draws.pop() 

    elif key == ord('b') and list_of_draws:  # Tecla 'z'
        print("Funciona 'b'")
        if list_of_draws[-1][0] == -1:
            list_of_draws.pop()
        while list_of_draws and list_of_draws[-1][0] != -1:
            list_of_draws.pop()

    # Si se pulsa 'd' cambiar de color
    elif key  == ord('d'):
      current_time = time.time()
      if current_time - last_d_key_press_time >= 1.1:
          current_color_index = (current_color_index +1) % len(color_names)
          last_d_key_press_time = current_time

    # Terminar si se pulsa 'q'
    elif cv2.waitKey(1) & 0xFF == ord('q'):
      break
cap.release()
cv2.destroyAllWindows()